# Taxonomy

## Dataset preparation

There are three subsets of data. They correspond to filter by title, body, and tags. 
1. posts_by_title: Posts containing "Github Actions" or its variants in the question title (github actions, github-action, Github actions, Github-actions, Github Actions, Github-Actions).
2. posts_by_body: Posts containing "Github Actions" or its variants in the question body.
3. posts_by_tags: Posts tagged with any of of github actions tag list ('github-actions', 'building-github-actions', 'github-actions-self-hosted-runners', 'github-actions-runners', 'github-actions-services', 'github-actions-artifacts', 'github-actions-reusable-workflows', 'github-actions-workflows', 'github-actions-marketplace')

In [1]:
import pandas as pd
import numpy as np

In [2]:
posts_by_title = pd.read_json('data/posts_by_title.json')
posts_by_body = pd.read_json('data/posts_by_body.json')
posts_by_tags = pd.read_json('data/posts_by_tags.json')
posts_by_title.rename(columns={"CONCAT('https://stackoverflow.com/q/', p.Id)": 'link'}, inplace=True)
posts_by_body.rename(columns={"CONCAT('https://stackoverflow.com/q/', p.Id)": 'link'}, inplace=True)
posts_by_tags.rename(columns={"CONCAT('https://stackoverflow.com/q/', p.Id)": 'link'}, inplace=True)

In [3]:
posts_by_body = posts_by_body[posts_by_body['post_type_id']==1] # Keeping only post bodies that correspond to question bodies
# posts_by_tags.drop(columns=['DeletionDate', 'OwnerDisplayName', 'LastEditorDisplayName', 'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate'], inplace=True);

In [4]:
# posts_by_tags.rename(columns={'Id':'id', 'PostTypeId':'post_type_id', 'AcceptedAnswerId': 'accepted_answer_id', 'ParentId': 'parent_id', 'CreationDate':'creation_date',
#                                 'Score':'score', 'ViewCount':'view_count', 'Body':'post_body', 'OwnerUserId':'owner_user_id', 'LastEditorUserId':'last_editor_user_id',
#                                 'LastEditDate':'last_edit_date', 'LastActivityDate':'last_activity_date', 'Title':'post_title', 'Tags':'tags', 'AnswerCount':'answer_count',
#                                 'CommentCount':'comment_count', 'ContentLicense':'content_license', 'Unnamed: 23':'link'}, inplace=True)


In [5]:
posts_by_tags.describe()
#posts_by_title.describe()
# posts_by_body.describe()

,id,post_type_id,accepted_answer_id,parent_id,score,view_count,owner_user_id,last_editor_user_id,answer_count,comment_count
count,9.873000e+03,9873.0,3.670000e+03,0.0,9873.000000,9873.000000,9.801000e+03,4.271000e+03,9873.000000,9873.000000
mean,7.126080e+07,1.0,7.022708e+07,NaN,2.747392,2217.544718,8.707827e+06,6.605726e+06,0.981363,1.657753
std,5.184317e+06,0.0,5.424942e+06,NaN,11.666862,8988.151347,6.554179e+06,6.039375e+06,1.119418,2.394376
min,5.417676e+07,1.0,5.417763e+07,NaN,-6.000000,5.000000,9.100000e+01,-1.000000e+00,0.000000,0.000000
25%,6.796411e+07,1.0,6.622763e+07,NaN,0.000000,136.000000,2.628868e+06,1.623876e+06,0.000000,0.000000
50%,7.263306e+07,1.0,7.129186e+07,NaN,1.000000,459.000000,7.764329e+06,4.290962e+06,1.000000,1.000000
75%,7.562319e+07,1.0,7.493441e+07,NaN,2.000000,1385.000000,1.383621e+07,1.023495e+07,1.000000,2.000000
max,7.759369e+07,1.0,7.758567e+07,NaN,348.000000,321529.000000,2.302257e+07,2.296569e+07,35.000000,26.000000


In [6]:
df_union = pd.concat([posts_by_tags, posts_by_title, posts_by_body]).drop_duplicates().reset_index(drop=True)
df_union.describe()

,id,post_type_id,accepted_answer_id,parent_id,score,view_count,owner_user_id,last_editor_user_id,answer_count,comment_count
count,1.132300e+04,11323.0,4.140000e+03,0.0,11323.000000,11323.000000,1.124500e+04,4.915000e+03,11323.000000,11323.000000
mean,7.133022e+07,1.0,7.028556e+07,NaN,2.563543,2062.098296,8.737486e+06,6.706649e+06,0.963967,1.648238
std,5.145350e+06,0.0,5.374074e+06,NaN,11.054158,8502.726944,6.573781e+06,6.117352e+06,1.085275,2.396634
min,5.417676e+07,1.0,5.417763e+07,NaN,-6.000000,5.000000,9.100000e+01,-1.000000e+00,0.000000,0.000000
25%,6.805283e+07,1.0,6.637378e+07,NaN,0.000000,132.000000,2.628868e+06,1.611459e+06,0.000000,0.000000
50%,7.267653e+07,1.0,7.129904e+07,NaN,1.000000,438.000000,7.793017e+06,4.420967e+06,1.000000,1.000000
75%,7.565271e+07,1.0,7.495300e+07,NaN,2.000000,1299.000000,1.391782e+07,1.060695e+07,1.000000,2.000000
max,7.759369e+07,1.0,7.758567e+07,NaN,348.000000,321529.000000,2.302257e+07,2.296569e+07,35.000000,26.000000


In [7]:
df_merged_1 = pd.merge(posts_by_tags, posts_by_title, how='inner', on='id')
df_intersection = pd.merge(df_merged_1, posts_by_body, how='inner', on='id')
df_intersection.describe()

,id,post_type_id_x,accepted_answer_id_x,parent_id_x,score_x,view_count_x,owner_user_id_x,last_editor_user_id_x,answer_count_x,comment_count_x,...,comment_count_y,post_type_id,accepted_answer_id,parent_id,score,view_count,owner_user_id,last_editor_user_id,answer_count,comment_count
count,2.903000e+03,2903.0,1.140000e+03,0.0,2903.000000,2903.000000,2.882000e+03,1.272000e+03,2903.000000,2903.000000,...,2903.000000,2903.0,1.140000e+03,0.0,2903.000000,2903.000000,2.882000e+03,1.272000e+03,2903.000000,2903.000000
mean,7.050473e+07,1.0,6.938052e+07,NaN,3.831898,2881.829142,8.590357e+06,6.978242e+06,1.053049,1.714433,...,1.714433,1.0,6.938052e+07,NaN,3.831898,2881.829142,8.590357e+06,6.978242e+06,1.053049,1.714433
std,5.605103e+06,0.0,5.787363e+06,NaN,16.139532,11830.760186,6.426077e+06,6.081297e+06,1.377158,2.484013,...,2.484013,0.0,5.787363e+06,NaN,16.139532,11830.760186,6.426077e+06,6.081297e+06,1.377158,2.484013
min,5.417676e+07,1.0,5.417763e+07,NaN,-3.000000,6.000000,9.100000e+01,-1.000000e+00,0.000000,0.000000,...,0.000000,1.0,5.417763e+07,NaN,-3.000000,6.000000,9.100000e+01,-1.000000e+00,0.000000,0.000000
25%,6.637150e+07,1.0,6.472536e+07,NaN,0.000000,183.000000,2.628868e+06,2.087967e+06,0.000000,0.000000,...,0.000000,1.0,6.472536e+07,NaN,0.000000,183.000000,2.628868e+06,2.087967e+06,0.000000,0.000000
50%,7.186596e+07,1.0,7.041612e+07,NaN,1.000000,570.000000,7.648607e+06,4.900238e+06,1.000000,1.000000,...,1.000000,1.0,7.041612e+07,NaN,1.000000,570.000000,7.648607e+06,4.900238e+06,1.000000,1.000000
75%,7.542059e+07,1.0,7.450144e+07,NaN,3.000000,1650.000000,1.352424e+07,1.106587e+07,1.000000,3.000000,...,3.000000,1.0,7.450144e+07,NaN,3.000000,1650.000000,1.352424e+07,1.106587e+07,1.000000,3.000000
max,7.759154e+07,1.0,7.758567e+07,NaN,348.000000,321529.000000,2.302257e+07,2.261377e+07,35.000000,26.000000,...,26.000000,1.0,7.758567e+07,NaN,348.000000,321529.000000,2.302257e+07,2.261377e+07,35.000000,26.000000


### Number of questions filtered:
- posts_by_title: 4538
- posts_by_body: 6297
- posts_by_tags: 9873
- df_intersection: 2903
- df_union: 11323

In [8]:
df_intersection['tags'].str.contains('<github-actions>').value_counts()

tags
True     2894
False       9
Name: count, dtype: int64

- We have decided to use df_intersection because we consider that we are ensuring that the topic is for sure related to GitHub Actions because it is tagged and mentioned in the title and the body question. 

- We decided to select a random sample.

- 340 posts or more are needed to have a confidence level of 95% that the real value is within ±5%.

In [9]:
# Creating the random sample. We are accepting the first 340 posts that we consider are related to the topc.
sample_size = 400
seed = 0
df_sample = df_intersection.sample(n=sample_size, random_state=seed)
print("Sample length: ", df_sample.shape[0])
df_sample.head()

Sample length:  400


,id,post_type_id_x,accepted_answer_id_x,parent_id_x,creation_date_x,score_x,view_count_x,post_body_x,owner_user_id_x,last_editor_user_id_x,...,owner_user_id,last_editor_user_id,last_edit_date,last_activity_date,post_title,tags,answer_count,comment_count,content_license,link
582,65011370,1,NaN,NaN,2020-11-25 19:17:14,0,637,<p>I have this go.yml for github actions</p>&#...,12215821.0,NaN,...,12215821.0,NaN,None,2020-11-26 08:31:42,"Github actions, problem with dep installing",<go><github-actions><dep>,1.0,0,CC BY-SA 4.0,https://stackoverflow.com/q/65011370
1811,73822327,1,73822678.0,NaN,2022-09-23 02:09:56,1,132,<p>I can locally create and use a docker conta...,7628816.0,7628816.0,...,7628816.0,7628816.0,2022-09-23 03:25:16,2022-09-23 03:25:37,ory/dockertest not working on GitHub Actions,<docker><github-actions><clickhouse><ory>,1.0,0,CC BY-SA 4.0,https://stackoverflow.com/q/73822327
2249,75607496,1,NaN,NaN,2023-03-01 18:12:01,1,107,<p>We have a legacy Maven application which is...,3065868.0,3065868.0,...,3065868.0,3065868.0,2023-03-03 08:57:38,2023-03-03 08:57:38,GitHub Actions: Generate Deployment sources fo...,<maven><github-actions><websphere><websphere-8...,0.0,9,CC BY-SA 4.0,https://stackoverflow.com/q/75607496
1652,73018888,1,NaN,NaN,2022-07-18 07:48:42,2,1437,<p>GitHub Actions concurrency broke my process...,119790.0,NaN,...,119790.0,NaN,None,2022-10-03 23:31:28,How to disable GitHub Actions Concurrency,<github><github-actions>,1.0,2,CC BY-SA 4.0,https://stackoverflow.com/q/73018888
667,65855054,1,NaN,NaN,2021-01-23 02:10:44,3,5595,<p>So I have a repo with multiple directories ...,3681199.0,NaN,...,3681199.0,NaN,None,2021-01-23 08:11:48,Can you have multiple working directories with...,<go><github-actions>,1.0,2,CC BY-SA 4.0,https://stackoverflow.com/q/65855054


In [27]:
df_sample[['id', 'link']].assign(evaluator1=np.nan).to_excel('data/sample_eval1.xlsx')
df_sample[['id', 'link']].assign(evaluator2=np.nan).to_excel('data/sample_eval2.xlsx')

In [28]:
sample_reviewed=pd.read_excel('data/sample_eval1_reviewed.xlsx')
sample_reviewed['evaluator2']=pd.read_excel('data/sample_eval2_reviewed.xlsx')['evaluator2']

In [45]:
sample_accepted=sample_reviewed[(sample_reviewed['evaluator1']==1)&(sample_reviewed['evaluator2']==1)].head(340)

In [47]:
pd.merge(df_intersection,sample_accepted[['id']], on='id', how='inner').to_excel('data/sample_accepted.xlsx')